# Remove unavailable videos and change names (uncommented)

In [ ]:
import json
import os
import cv2
from pathlib import Path

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
json_file = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/WLASL_v0.3.json"

with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

video_labels = {}
for entry in data:
    gloss = entry["gloss"]
    for instance in entry["instances"]:
        video_id = str(instance["video_id"])  
        if video_id not in video_labels:
            video_labels[video_id] = []
        video_labels[video_id].append(gloss)

video_folder = Path(video_folder)

if not video_folder.exists():
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = list(video_folder.glob("*"))


deleted_videos = []
renamed_videos = []

def is_video_valid(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return False  
    ret, _ = cap.read()
    cap.release()
    return ret 

for video_file in video_files:
    if not video_file.is_file():
        continue

    video_id = video_file.stem 

    if os.path.getsize(video_file) == 0 or not is_video_valid(video_file):
        deleted_videos.append(video_file.name)
        os.remove(video_file)
        continue

    # labels = video_labels.get(video_id, [])
    # if labels:
    #     new_name = f"{video_id}_{'_'.join(labels)}{video_file.suffix}"
    #     new_path = video_file.parent / new_name
    #     os.rename(video_file, new_path)
    #     renamed_videos.append((video_file.name, new_name))

print("Gelöschte Videos:", deleted_videos)
print("Umbenannte Videos:", renamed_videos)


# Add frame count to video names

In [ ]:
import cv2
from pathlib import Path

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

def get_frame_count(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None 
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

renamed_videos = []

for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    video_id, ext = os.path.splitext(video_file) 
    
    frame_count = get_frame_count(video_path)
    
    if frame_count is not None:
        new_name = f"{video_id}_{frame_count}{ext}"
        new_path = os.path.join(video_folder, new_name)
        os.rename(video_path, new_path)
        renamed_videos.append((video_file, new_name))

print("Umbenannte Videos mit Frames:", renamed_videos)

# Informations about videos

In [ ]:
import os
from collections import Counter

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

label_counter = Counter()

for video_file in video_files:
    filename, _ = os.path.splitext(video_file) 
    parts = filename.split("_") 

    if len(parts) == 3:  
        label = parts[1]  
        label_counter[label] += 1

print(f"Anzahl der verschiedenen Labels: {len(label_counter)}")
print("Top 10 häufigste Labels:")
for label, count in label_counter.most_common(10):
    print(f"{label}: {count}")


In [ ]:
# Gruppen nach Anzahl der Videos pro Label
from collections import Counter

label_distribution = Counter(label_counter.values())

print("Anzahl der Labels mit bestimmter Anzahl von Videos:")
for num_videos, num_labels in sorted(label_distribution.items()):
    print(f"{num_labels} Labels haben {num_videos} Videos")

# Delete videos with less than min_videos_per_label

In [ ]:
import os
from collections import Counter

# Verzeichnis mit den Videos
video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

# Liste aller Videos abrufen
video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

# Labels extrahieren und zählen
label_counter = Counter()

for video_file in video_files:
    filename, _ = os.path.splitext(video_file)  # Entfernt Dateiendung
    parts = filename.split("_")  # Trennt nach "_"
    
    if len(parts) >= 3:  # Sicherstellen, dass das Format korrekt ist
        label = parts[1]  # Mittleres Element ist das Label
        label_counter[label] += 1

# Mindestanzahl an Videos pro Label
min_videos_per_label = 8

# Dateien mit zu wenigen Videos löschen
deleted_count = 0

for video_file in video_files:
    filename, _ = os.path.splitext(video_file)
    parts = filename.split("_")

    if len(parts) >= 3:
        label = parts[1]
        if label_counter[label] < min_videos_per_label:
            os.remove(os.path.join(video_folder, video_file))
            deleted_count += 1

print(f"{deleted_count} Videos wurden gelöscht, da ihr Label weniger als {min_videos_per_label} Videos hatte.")

---> start here

# Check Frames again

In [ ]:
import os
import cv2
import re

# Verzeichnis mit den Videos
video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"❌ Der Ordner {video_folder} wurde nicht gefunden.")

# Funktion zur Bestimmung der aktuellen Frame-Anzahl
def get_frame_count(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None 
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

# Regex, um die Frame-Anzahl im Dateinamen zu finden (z.B. `00624_accident_109_bright.mp4`)
frame_pattern = re.compile(r"_(\d+)(?:_[a-zA-Z]+)?\.mp4$")

renamed_videos = []

for video_file in os.listdir(video_folder):
    if not video_file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, video_file)

    # Extrahiere die tatsächliche Frame-Anzahl
    frame_count = get_frame_count(video_path)
    if frame_count is None:
        print(f"⚠️ Warnung: Konnte Frames für {video_file} nicht bestimmen. Überspringe...")
        continue

    # Prüfe, ob bereits eine Frame-Anzahl im Namen existiert
    match = frame_pattern.search(video_file)
    
    if match:
        existing_frame_count = int(match.group(1))
        
        # Falls die Frame-Anzahl nicht übereinstimmt, umbenennen
        if existing_frame_count != frame_count:
            new_name = re.sub(frame_pattern, f"_{frame_count}.mp4", video_file)
            new_path = os.path.join(video_folder, new_name)
            os.rename(video_path, new_path)
            renamed_videos.append((video_file, new_name))
            print(f"🔄 {video_file} → {new_name} (Frames aktualisiert)")
    else:
        # Falls keine Frame-Anzahl vorhanden ist, hinzufügen
        name_parts = video_file.rsplit(".", 1)
        new_name = f"{name_parts[0]}_{frame_count}.mp4"
        new_path = os.path.join(video_folder, new_name)
        os.rename(video_path, new_path)
        renamed_videos.append((video_file, new_name))
        print(f"➕ {video_file} → {new_name} (Frames hinzugefügt)")

print("\n✅ Alle Frame-Anzahlen wurden überprüft und ggf. aktualisiert!")

# Print videos with less than 40 min_videos_per_label

In [ ]:
import os
from collections import Counter

# Verzeichnisse mit den Videos
raw_video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Labels zählen
label_counter = Counter()

for folder in [raw_video_folder, augmented_video_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        parts = video_file.split("_")
        if len(parts) >= 3:  # Sicherstellen, dass der Name korrekt ist
            label = parts[1]  # Das zweite Element ist jetzt das Label
            label_counter[label] += 1

# Sortiere die Labels nach Anzahl der Videos (absteigend)
sorted_labels = sorted(label_counter.items(), key=lambda x: x[1], reverse=True)

# Anzahl der verfügbaren Labels
print(f"Anzahl der verbleibenden Labels: {len(sorted_labels)}")

# Top 10 häufigste Labels ausgeben
print("\nTop 10 häufigste Labels:")
for label, count in sorted_labels[:10]:
    print(f"{label}: {count}")

# Alle Labels mit Anzahl der Videos ausgeben
print("\nAnzahl der Videos pro Label:")
for label, count in sorted_labels:
    print(f"{label}: {count}")

# Calculate maximum number of frames

In [ ]:
import os
import torch

# Verzeichnisse mit Videos
original_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Bestimme die maximale Anzahl an Frames über beide Ordner
max_frames = 0

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        parts = video_file.replace(".mp4", "").split("_")
        
        if len(parts) >= 3:  # Sicherstellen, dass genug Teile vorhanden sind
            try:
                frame_count = int(parts[2])  # Die Frame-Anzahl ist an Position 2
                max_frames = max(max_frames, frame_count)
            except ValueError:
                print(f"⚠️ Warnung: Konnte Frame-Anzahl nicht extrahieren aus {video_file}")

print(f"✅ Maximale Anzahl an Frames (über alle Videos): {max_frames}")

# Reduce number of videos per label

In [ ]:
import os
import random
from collections import Counter

# Verzeichnisse mit Videos
original_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Zielanzahl an Videos pro Label
target_videos_per_label = 36

# Zähle die Anzahl der Videos pro Label
label_counter = Counter()

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        parts = video_file.split("_")
        if len(parts) >= 3:
            label = parts[1]  # Das Label ist das zweite Element
            label_counter[label] += 1

# Überprüfung, ob Labels weniger als 48 Videos haben
for label, count in label_counter.items():
    if count < target_videos_per_label:
        print(f"⚠️ WARNUNG: Label '{label}' hat nur {count} Videos und kann nicht auf 48 erhöht werden!")

# Entferne überschüssige augmentierte Videos zufällig
for label, count in label_counter.items():
    if count > target_videos_per_label:
        # Finde alle augmentierten Videos für dieses Label
        label_videos = [f for f in os.listdir(augmented_folder) if f.split("_")[1] == label]

        # Falls es nicht genug augmentierte Videos gibt, warnen
        if len(label_videos) < (count - target_videos_per_label):
            print(f"⚠️ WARNUNG: Nicht genug augmentierte Videos zum Löschen für Label '{label}'!")

        # Wähle zufällig die zu löschenden Videos (nur augmentierte)
        remove_videos = random.sample(label_videos, count - target_videos_per_label)

        # Lösche die Videos
        for video_file in remove_videos:
            os.remove(os.path.join(augmented_folder, video_file))
            print(f"🗑️ Gelöscht: {video_file}")

print("✅ Alle Labels haben jetzt genau 36 Videos!")

# Normalize frames

In [ ]:
import os
import cv2
import torch
import numpy as np

# Verzeichnisse
original_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"
processed_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_processed"
os.makedirs(processed_folder, exist_ok=True)

# Bestimme die maximale Anzahl an Frames falls nicht definiert
if "max_frames" not in locals():
    max_frames = 195  # Standardwert, falls nicht zuvor berechnet

# Funktion zur Extraktion von Frames als Torch-Tensoren
def extract_frames(video_path, device="cuda"):
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        print(f"⚠️ Warnung: Konnte Video nicht öffnen: {video_path}")
        return torch.zeros((1, 3, 224, 224), dtype=torch.float32, device=device)  # Dummy-Frame
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))  # Frames auf gleiche Größe bringen
        frame = torch.tensor(frame, dtype=torch.float32, device=device).permute(2, 0, 1)  # [H, W, C] → [C, H, W]
        frames.append(frame)

    cap.release()

    if not frames:
        print(f"⚠️ Warnung: Keine Frames extrahiert für {video_path}")
        return torch.zeros((1, 3, 224, 224), dtype=torch.float32, device=device)  # Falls leer, Dummy-Frame

    return torch.stack(frames, dim=0)

# Funktion zum Padding mit GPU-Unterstützung
def pad_frames(frames, target_length, device="cuda"):
    num_frames = frames.shape[0]

    if num_frames < target_length:
        padding = torch.zeros((target_length - num_frames, 3, 224, 224), dtype=torch.float32, device=device)
        return torch.cat((frames, padding), dim=0)
    else:
        return frames[:target_length]  # Falls zu lang, einfach abschneiden

# Verarbeitung aller Videos (Original + Augmented)
device = "cuda" if torch.cuda.is_available() else "cpu"

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        video_path = os.path.join(folder, video_file)

        try:
            # Frames extrahieren und auf GPU laden
            frames = extract_frames(video_path, device=device)

            # Padding / Kürzen auf `max_frames`
            padded_frames = pad_frames(frames, max_frames, device=device)

            # Speichern als `.npy`-Datei (wieder auf CPU konvertieren)
            npy_path = os.path.join(processed_folder, video_file.replace(".mp4", ".npy"))
            np.save(npy_path, padded_frames.cpu().numpy())

            print(f"✅ {video_file} verarbeitet und gespeichert als {npy_path}")

        except Exception as e:
            print(f"❌ Fehler beim Verarbeiten von {video_file}: {e}")

print("🚀 GPU-basierte Normalisierung abgeschlossen!")